In [1]:
# QPIE method ( Quantum Probability Image Encoding )

# Importing the libraries
import numpy as np
import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, BasicAer, transpile, assemble
from qiskit.execute_function import execute
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_histogram
from PIL import Image
from typing import Dict, List, Tuple
from collections import Counter
from sklearn.metrics import mean_squared_error
from qiskit.aqua.components.feature_maps import FeatureMap
import pickle
import json
from itertools import chain
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# GLOBAL VARIABLES
DIM = 28
NUM_QUBITS = 10


###########################
##### Functions #####
###########################

def encode_img(image, register_num):
    ''' encoding of image using QPIE method '''
    img = list(chain(*image))
    pix_val = img

    # normalize
    pix_norm = np.linalg.norm(pix_val)
    pix_val = np.array(pix_val)
    arr_norm = pix_val/pix_norm
    arr_norm = arr_norm.tolist()

    # Encode onto the quantum register
    qc = QuantumCircuit(register_num)
    # test = arr_norm.append(np.zeros(2**10-arr_norm.shape))
    test = arr_norm + np.zeros(2**register_num-DIM**2).tolist()
    qc.initialize(test, qc.qubits)
    return qc


def encode(image):
    ''' final wrapper function (for submission) '''
    return encode_img(255*255*image, register_num=NUM_QUBITS)


# apply qft

def apply_qft(circuit, register_num):
    circuit.append(
        QFT(register_num, do_swaps=False).to_gate(), circuit.qubits)
    return circuit


################
# Helpers
################


def simulate(circuit: QuantumCircuit) -> Dict:
    """Simulate circuit given state vector"""
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circuit, backend)
    result = job.result()
    state_vector = result.get_statevector(circuit)

    histogram = dict()
    for i in range(len(state_vector)):
        population = abs(state_vector[i]) ** 2
        if population > 1e-9:
            histogram[i] = population

    return histogram


def histogram_to_cat(histogram):
    assert abs(sum(histogram.values()) - 1) < 1e-8
    positive = 0
    for key in histogram.keys():
        digits = bin(int(key))[2:].zfill(20)
        if digits[-1] == '0':
            positive += histogram[key]

    return positive


def image_mse(img1, img2):
    return mean_squared_error(img1, img2)


def count_gates(circuit: qiskit.QuantumCircuit) -> Dict[int, int]:
    """ finds num of gate operations with each num of qubits """
    return Counter([len(gate[1]) for gate in circuit.data])



C:\Users\pc\AppData\Local\Programs\Python\Python39\lib\site-packages\qiskit\aqua\__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


In [2]:
class CustomFeatureMap(FeatureMap):
    """Mapping data with a custom feature map."""
    
    def __init__(self, feature_dimension, depth=2, entangler_map=None):
        """
        Args:
            feature_dimension (int): number of features
            depth (int): the number of repeated circuits
            entangler_map (list[list]): describe the connectivity of qubits, each list describes
                                        [source, target], or None for full entanglement.
                                        Note that the order is the list is the order of
                                        applying the two-qubit gate.        
        """
        self._support_parameterized_circuit = False
        self._feature_dimension = feature_dimension
        self._num_qubits = self._feature_dimension = feature_dimension
        self._qc = None
        self._depth = depth
        self._entangler_map = None
        if self._entangler_map is None:
            self._entangler_map = [[i, j] for i in range(self._feature_dimension) for j in range(i + 1, self._feature_dimension)]

    def apply_qft(self, circuit, register_num):
        circuit.append(
            QFT(register_num, do_swaps=False).to_gate(), circuit.qubits)
        return circuit

    def encode_img(self, image, register_num):
        ''' encoding of image using QPIE method '''
#         print(image)
#         print(type(image))
#         print(image.shape)
        img = list(chain(*image))
        pix_val = img
        

        # normalize
        pix_norm = np.linalg.norm(pix_val)
        pix_val = np.array(pix_val)
        arr_norm = pix_val/pix_norm
        arr_norm = arr_norm.tolist()

        # Encode onto the quantum register
        qc = QuantumCircuit(register_num)
        # test = arr_norm.append(np.zeros(2**10-arr_norm.shape))
        test = arr_norm + np.zeros(2**register_num-DIM**2).tolist()
        qc.initialize(test,qc.qubits)
        return qc

    def encode(self, image):
        ''' final wrapper function (for submission) '''
#         return self.encode_img(255*255*image, register_num=NUM_QUBITS)
        print("")
        return self.apply_qft(self.encode_img(255*255*image, register_num=NUM_QUBITS), register_num=NUM_QUBITS)
    
    def getQC():
        return self.qc

    def construct_circuit(self, x, qr, inverse=False):
            """Construct the feature map circuit.
            
            Args:
                x (numpy.ndarray): 1-D to-be-transformed data.
                qr (QauntumRegister): the QuantumRegister object for the circuit.
                inverse (bool): whether or not to invert the circuit.
                
            Returns:
                QuantumCircuit: a quantum circuit transforming data x.
            """
            qc = self.encode(x)
                        
            if inverse:
                qc.inverse()
                
            self._qc = qc
            return qc

In [8]:
###########################
##### Main #####
###########################

# load data
images = np.load('../data/images.npy')
labels = np.load('../data/labels.npy')

# encoding
# circuit = encode_img(255*255*images[550], n)
# # apply classifier
# # circuit = apply_qft(circuit, n)
# histogram = simulate(circuit)
# # state vector: keys are the pixel position and the values are the normalized grayscale value. pixels with 0 grayscale are left out
# plot_histogram(histogram)
# # qft_label = histogram_to_cat(histogram)
# # print(qft_label)
# # decoding
# fig, ax = plt.subplots(1)
# ax.set_aspect('equal')
# # this is the array you want to plot
# array_plot = decode_img(histogram)
# plt.imshow(array_plot, interpolation='nearest', cmap=plt.cm.hsv)
# plt.gray()
# u = plt.colorbar(fraction=0.046, pad=0.04)
# plt.clim(0, np.max(array_plot))
#
#
# plt.xlim([0, DIM])
# plt.ylim([0, DIM])
#
# # set fig size
# fig.set_size_inches(5, 5)
# fig.set_dpi(100)
# # and ticks size
# plt.xticks(fontsize=22)
# plt.yticks(fontsize=22)
# plt.gca().invert_yaxis()
# plt.show()


# test part2
def run_part2(image,):
    
    with open('quantum_classifier_10samples.pkl', 'rb') as f:
        classifier = pickle.load(f)
        f.close()
        
#     with open('quantum_classifier_10_Modified_samples.pkl', 'rb') as f:
#         classifierM = pickle.load(f)
#         f.close()
     
#    circuit = encode(image)

#    qc = classifier.encode(image)
#    print(type(classifier))

#     circuit.append(classifierM.getQC())
     
#     histogram = simulate(circuit)
#     qc1 = encode(image)
#     qc1.append(encode(image))

    circuit = classifier.feature_map.construct_circuit(image,None)
    histogram = simulate(circuit)
    label_pred = histogram_to_cat(histogram)

    # theshold can be decided to be whatever you want
    threshold = 0.5
    if label_pred > threshold:
        label_pred = True # T-Shirt Image
    else:
        label_pred = False # Not a T-Shirt Image

    return circuit, label_pred




In [9]:
start = 0
stop = 5
# images_ = images[start:stop]
length = stop - start
gatecount = 0
score = 0

for i in range(start,stop):
    # encode
    circuit, label_pred = run_part2(images[i])
#     print(circuit)
#     print(circuit.data)
    print(i,"Prediction:",label_pred,", Actual:",labels[i])
    # count num of 2qubit gates
    gatecount += count_gates(circuit)[2]
    
    
   #check label
    if label_pred==labels[i]:
        score+=1
        
#score
score=score/length
gatecount=gatecount/length

print("\nAccuracy = %.2f%%"%(score*100))
print("Challenge Score:",score*(0.999**gatecount))


0 Prediction: True , Actual: True

1 Prediction: True , Actual: True

2 Prediction: True , Actual: True

3 Prediction: True , Actual: True

4 Prediction: True , Actual: True

Accuracy = 100.00%
Challenge Score: 1.0
